# Area Processing (Tutorial 2)
***
# Table of Contents
1.   [Imports](#Imports)
2.   [Image Analysis](#Images-Analysis)
3.   [Exercise 1 - Sliding Window](#Exercise-1---Sliding-Window)
4.   [Exercise 2 - Convolution on RoI](#Exercise-2---Convolution-on-RoI)
5.   [Exercise 3 - Convolution on the Whole Image](#Exercise-3---Convolution-on-the-Whole-Image)
6.   [Exercise 4 - Different Convolution Kernels](#Exercise-4---Different-Convolution-Kernels)

# Imports

Only 4 libraries are needed for this project:
* opencv (cv2) - For image processing
* numpy - For its arrays
* matplotlib - Plotting histograms
* os - File traversal
* tqdm.notebook - tqdm progress bars, but for ipynb files
* Classes - Custom classes written by me for this assignment

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from tqdm.notebook import tqdm
from Classes import Window, Sobel, Gaussian, Bilinear

# Image Analysis

### aiden dp.jpg

This is a picture of me in the upper barakka gardens in Valletta. The focus is myself with the background of the Grand
Harbour. The picture is lighted by natural lighting and includes a lot of non linear features.

<img src="images/aiden dp.jpg">

### cursed.jpg

The image I will be using for this lab is one of my failed attempts of creating a pie chart for one of my statistics units
from last year. It features a sun like object in the centre with black lines coming out from the centre, it is coloured in
full RGB while the background is gray.

<img src="images/cursed.jpg">

### dog chaos.png

This is a blurry picture of two of my friends and another friend's dog. They are in a dimly lit, tiled floor room with a
white wall and some furniture behind one of my friends.

<img src="images/dog chaos.png">

### jake car.png

This is a picture of my classmate Jake standing next to a red car. The photo was taken in a street during daytime. The
background features several home facades, 2 other cars and a person.

<img src="images/jake car.png">

### jake close up.jpg

This is a picture of my classmate Jake with a beige background.

<img src="images/jake close up.jpg">

### jake sitting.jpg

This is a picture of my classmate Jake sitting on the floor of a bathroom. The room is well lit and tiled all over.

<img src="images/jake sitting.jpg">

### jojo ben.jpg

This is a picture of my classmate Ben walking towards the Hal Ghaxaq church with a bag of fried chicken in his hand. The
photo was taken during the night so the lighting comes from old street lamps.

<img src="images/jojo ben.jpg">

#### I have permission by all the people shown to use these images for this tutorial

Here I load the images into a list

In [2]:
raw_images = {}
for file in tqdm(os.listdir("images"), desc='Loading Images'):
    raw_images[file] = cv2.imread("images/" + file)

Loading Images:   0%|          | 0/7 [00:00<?, ?it/s]

# Exercise 1 - Sliding Window

### Window Class

For this exercise I wrote the window class where I define some properties for the window.

```python
def __init__(self, image, n, s):
    self.x_boundary = image.shape[1] + n
    self.y_boundary = image.shape[0] + n
    self.top_left = (0, 0)
    self.bot_right = (n, n)
    self.previousBotY = n
    self.height = n
    self.stride = (s, s)
    try:
        self.channels = image.shape[2]
    except:
        self.channels = 1
```

To create a Window, the image (min 2d numpy array), n (length or width) and s(stride or step) are required.

Using these parameters I define:
* The x and y boundaries for the window.
* The starting top left and bottom right location as a tuple of two positions, where [0] is x and [1] is y.
* Previous y position, this is used to check if the window has changed it's y position.
* Height
* Stride
* Number of channels

Each window is a square. The object is intended to be used for the image passed in initialisation.

### Using Window for Ex 1

For this exercise I use 4 functions from the class:

```Python
def getPos(self):
    return self.top_left, self.bot_right
```

getPos returns the current position of the Window

```python
def forwardPos(self):
    # Case when you need to go down and start new line
    if (self.bot_right + self.stride)[0] >= (self.x_boundary - self.height):
        return (0, self.top_left[1] + self.stride[1]), (self.height, self.bot_right[1] + self.stride[1])
    # Generic move right case
    else:
        return (self.top_left[0] + self.stride[0], self.top_left[1]), \
               (self.bot_right[0] + self.stride[0], self.bot_right[1])
```

forwardPos returns the would be position of the next move. There are two cases:
1. Next step stays in X boundary and so the new positions are just changed by adding stride
2. Special case when next step would exceed X boundary so x positions are reset to 0, n and y positions are incremented
by stride

```python
def forwardMove(self):
    # Change positions
    self.top_left, self.bot_right = self.forwardPos()
    return self.top_left, self.bot_right
```

forwardMove changes the window's position to the return of forwardPos

```python
def inBoundary(self, new_top_left=None, new_bot_right=None):
    # Use current position if no new positions are passed
    if new_top_left is None:
        new_top_left = self.top_left
    if new_bot_right is None:
        new_bot_right = self.bot_right
    # Check if parameters are in boundary of the image given in initialisation
    return new_bot_right[0] <= self.x_boundary and new_bot_right[1] <= self.y_boundary and \
           new_top_left[0] >= 0 and new_top_left[1] >= 0
```

inBoundary returns whether given positions, or the current positions are inBoundary of the image.


### Code Explanation

First I initialise a Window for 'aiden dp.png' n=100, s=50. The reason for it being quite a "large" window is so that
the sliding window demonstration can go fast.

A rectangle is drawn over the initial positions and saved.

rectangles are drawn in red.

The starting and future positions are read using getPos and forwardPos. It is expected that win is initialised in boundary.

Then I loop while win is in its boundary.

* Every iteration I draw a rectangle on the image using cv2.rectangle.
* If show is on I display this using imshow.
* Then the positions are moved forwardMove and the future positions are taken again using forwardPos.

In [3]:
%%capture
# On/Off switch
show = False

win = Window(raw_images["aiden dp.png"], 100, 50)
start_point, end_point = win.getPos()

image = cv2.rectangle(raw_images["aiden dp.png"].copy(), start_point, end_point, (0, 0, 255))
cv2.imwrite("Output/aidenrectangle.png", image, [cv2.IMWRITE_PNG_COMPRESSION, 0])

new_tl, new_br = win.forwardPos()
while win.inBoundary(new_br):
    image = cv2.rectangle(raw_images["aiden dp.png"].copy(), start_point, end_point, (0, 0, 255))
    if show:
        cv2.imshow("Sliding Window", image)
        cv2.waitKey(int(1/35*1000))
    start_point, end_point = win.forwardMove()
    new_tl, new_br = win.forwardPos()
cv2.destroyAllWindows()

A rectangle drawn using cv2.rectangle with the bounds taken from the Window object.

<img src="Output/aidenrectangle.png">

# Exercise 2 - Convolution on RoI

### Kernel Class

For this exercise I wrote the window class where I define some properties for the window.

```python
def __init__(self, kernel, weight):
    self.kernel = kernel
    self.weight = weight
```

To create a Kernel, its kernel (numpy array) and weight  are required.

Using these parameters I define:
* The kernel which will be used to multiply and sum over a given area.
* The weight which will be multiplied to the result of the kernel pass.

Like the Window class, each Kernel is a square.


### Using Kernel for Remaining Exercises

For the remaining exercises I use the 2 functions from the class:

```Python
def filter(self, roi, axis=0, channels=1):
    ret = []
    if axis == 2:
        for i in range(channels):
            if channels == 1:
                _filter = self.kernel * roi
            else:
                _filter = self.kernel * roi[:, :, i]
            sum_of_filter1 = _filter.sum()
            if channels == 1:
                _filter = self.kernel.T * roi
            else:
                _filter = self.kernel.T * roi[:, :, i]
            sum_of_filter2 = _filter.sum()
            ret.append((((sum_of_filter1 ** 2) + (sum_of_filter2 ** 2)) ** (1 / 2)) * self.weight)

        return np.array(ret)
    else:
        for i in range(channels):
            if channels == 1:
                _filter = self.kernel * roi
            else:
                _filter = self.kernel * roi[:, :, i]
            ret.append(_filter.sum() * self.weight)

        return np.array(ret)
```

The filter function takes a roi, the axis of operation, and number channels of roi. There are two main parts and both
function similarly.

#### roi and kernel must have the same shape, otherwise there will be a shape error in the multiplication stage

When axis is 2:

1. Loop over each channel.
2. Multiply roi's pixels in the channel with the kernel.
3. Sum these values.
4. Multiply roi's pixels in the channel with the kernel's transpose.
5. Sum these values.
6. Get the magnitude of both these values by squaring, adding then getting their square root. (vector magnitude)
7. Multiply this result with kernel's weight.
8. Append each result of every channel to a list.
9. Convert ret to a numpy array and return.

When axis is not 2:

1. Loop over each channel.
2. Multiply roi's pixels in the channel with the kernel.
3. Sum these values.
4. Multiply this result with the kernel's weight.
5. Append each result of every channel to a list
6. Convert ret to a numpy array and return

In any case the return of filter is a pixel.


```Python
def filterImage(self, image, stride=1, window=None, axis=0):
    new_image = []
    line = []
    if window is None:
        moving_kernel = Window(image, self.kernel.shape[0], stride)
    else:
        image = window.getImageInBoundary(image)
        moving_kernel = Window(image, self.kernel.shape[0], stride)

    new_tl, _ = moving_kernel.forwardPos()
    while moving_kernel.inBoundary(new_tl):
        roi = moving_kernel.getImageInBoundary(image)
        if moving_kernel.changedY():
            new_image.append(line)
            line = []

        line.append(self.filter(roi, axis, moving_kernel.channels))

        moving_kernel.forwardMove()
        new_tl, _ = moving_kernel.forwardPos()

    return np.array(new_image)
```

The filterImage function takes an image, stride, a window if it is expected to function on a RoI, and the axis of
operation.

new_image and line are 2 lists i will use for this function. new_image will be the output of passing the kernel over the
image/RoI and line will be used to represent a  line of pixels.

The first thing I do is check whether window was defined or not. If it is defined then the function is expected to work
on a RoI, defined by window, on image and not the entire image. Hence, if it is defined I use the getImageInBoundary from
the Window Class (explained below) and assign image to it. In any case, a moving_kernel is defined for image, the kernel
and stride.

Similar to the way I move the rectangle in Ex 1 using the Window move functions, here I define a RoI using moving_kernel
and getImageInBoundary for image. This gives me a nxn copy of the image in moving_kernel's boundary. n here is the width
and height of the Kernel.

Then I check if the y position of moving_kernel has changed, if it did then I append line to new_image and reset line.
This should happen in the first iteration.

After this I append the filtered roi using the filter function to line.

The next steps relate to the moving of the kernel.

Finally new_images is returned as a numpy array.

### Using Window for Remaining Exercises

For this exercise I use 2 new functions (the other 4 functions are explained above) from the class:

```Python
def changedY(self):
    if self.previousBotY == self.bot_right[1]:
        return False
    else:
        self.previousBotY = self.bot_right[1]
        return True
```

changedY checks whether the y value of the bottom right corner of the window has changed or not.

```python
def getImageInBoundary(self, image):
    new_image = []
    for i in range(self.top_left[1], self.bot_right[1]):
        if i >= image.shape[0]:
            continue
        new_image.append(image[i][self.top_left[0]: self.bot_right[0]])

    if self.channels == 1:
        return np.resize(np.array(new_image), (self.height, self.height))
    else:
        return np.resize(np.array(new_image), (self.height, self.height, self.channels))
```

getImageInBoundary returns a numpy array of the pixels of image in the boundary of the Window. This is done by looping
from the y value of the top left corner to the y value of the bottom right corner. Then for each iteration, using list
slicing I append the x values from left to right.

The return is sized appropriate to the image's ndims size.

### Kernels

```python
class Sobel:
    def __init__(self, weight):
        self.kernel = Kernel(np.array([[-1, 0, 1],
                                       [-2, 0, 2],
                                       [-1, 0, 1]]),
                             weight)

    def filterImage(self, image, stride=1, window=None, axis=2):
        return self.kernel.filterImage(image, stride, window, axis)


class Gaussian:
    def __init__(self, size, weight):
        fwhm = size // 2
        x = np.arange(0, size, 1, float)
        y = x[:, np.newaxis]
        x0 = y0 = size // 2
        self.kernel = Kernel(np.exp(-4 * np.log(2) * ((x - x0) ** 2 + (y - y0) ** 2) / fwhm ** 2), weight)

    def filterImage(self, image, stride=1, window=None, axis=0):
        return self.kernel.filterImage(image, stride, window, axis)


class Bilinear:
    def __init__(self, weight):
        self.kernel = Kernel(np.array([[1, 2, 1],
                                       [2, 4, 2],
                                       [1, 2, 1]]),
                             weight)

    def filterImage(self, image, stride=1, window=None, axis=0):
        return self.kernel.filterImage(image, stride, window, axis)
```

The kernels used in this tutorial are then defined as above. The sobel and bilinear kernels are hard coded, while the
gaussian kernel uses the code provided in the lecture notes to generate a kernel for the given size. Weight is explained
above.

### Code Explanation

I initialise a sobel kernel, I give a neutral weight because I don't think it needs one.

In [4]:
sobel = Sobel(1)

### Code Explanation

I loop over every image and get a roi using getImageInBoundary of shape (300,300).
Then I filter this roi using the sobel(x+y) kernel.

In [5]:
for image in tqdm(raw_images):
    win = Window(raw_images[image], 300, 1)
    roi = win.getImageInBoundary(raw_images[image])
    cv2.imwrite("Output/RoI/"+ image +"_before_filter.png", roi, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    filtered = sobel.filterImage(image=raw_images[image], window=win, axis=2)
    cv2.imwrite("Output/RoI/"+ image +"_after_filter.png", filtered, [cv2.IMWRITE_PNG_COMPRESSION, 0])

  0%|          | 0/7 [00:00<?, ?it/s]

### Code Explanation

I copy the "aiden dp.png" image, apply grayscale to it then get a roi and filter it as above.

In [6]:
%%capture
image = raw_images["aiden dp.png"].copy()
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
win = Window(image, 300, 1)
roi = win.getImageInBoundary(image)
cv2.imwrite("Output/RoI/aidendp_before_filter.png", roi, [cv2.IMWRITE_PNG_COMPRESSION, 0])
filtered = sobel.filterImage(image=image, window=win)
cv2.imwrite("Output/RoI/aidendp_after_filter.png", filtered, [cv2.IMWRITE_PNG_COMPRESSION, 0])

### Results

# Exercise 3 - Convolution on the Whole Image

### Code Explanation

I loop over every image and apply the sobel(x+y) filter on them.

In [7]:
for image in tqdm(raw_images):
    filtered = sobel.filterImage(image=raw_images[image], axis=2)
    cv2.imwrite("Output/Full Image/Sobel2/"+ image +"_after_filter.png", filtered, [cv2.IMWRITE_PNG_COMPRESSION, 0])

  0%|          | 0/7 [00:00<?, ?it/s]

### Results

# Exercise 4 - Different Convolution Kernels

### Code Explanation

I initialise a bilinear kernel, I give it a weight of 1.8 because otherwise the image would be too bright.

In [8]:
bileaner = Bilinear(1/8)

### Code Explanation

I initialise a gaussian kernel, I give it a weight of 1.8 because otherwise the image would be too bright.


In [9]:
gaussian = Gaussian(5, 1/8)

### Code Explanation

I collect the filters into a dict to make the last step easier.

In [10]:
filters = {"Sobel":sobel,
           "Bilinear":bileaner,
           "Gaussian":gaussian}

### Code Explanation

I loop over every image and apply all the filters to them. However for the sobel kernel, instead of using the x+y here I
use x and y seperately.

In [11]:
for image in tqdm(raw_images):
    for filter in filters:
        if filter == "Sobel":
            filtered = filters[filter].filterImage(image=raw_images[image], axis=0)
            cv2.imwrite("Output/Full Image/Sobel0/"+ image +"_after_filter.png", filtered, [cv2.IMWRITE_PNG_COMPRESSION, 0])
            filtered = filters[filter].filterImage(image=raw_images[image], axis=1)
            cv2.imwrite("Output/Full Image/Sobel1/"+ image +"_after_filter.png", filtered, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        else:
            filtered = filters[filter].filterImage(image=raw_images[image])
            cv2.imwrite("Output/Full Image/"+ filter + "/" + image +"_after_filter.png", filtered, [cv2.IMWRITE_PNG_COMPRESSION, 0])

  0%|          | 0/7 [00:00<?, ?it/s]

### Results